# BCI AutoML Platform — Minimal Example

This notebook demonstrates:
1. Load dataset (synthetic or BCI IV 2a)
2. Trial-wise split (no leakage)
3. Run calibration
4. Show leaderboard
5. Select best pipeline
6. Snapshot saving

In [ ]:
import sys
from pathlib import Path

# Project root: run notebook from repo root (EEG Agent) or from examples/
ROOT = Path(".").resolve()
if (ROOT / "bci_framework").exists():
    pass
else:
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from bci_framework.utils.config_loader import load_config, get_config
from bci_framework.utils.experiment import set_seed, get_experiment_id
from bci_framework.utils.splits import get_train_test_trials
from bci_framework.datasets import get_dataset_loader
from bci_framework.datasets.synthetic_eeg import generate_synthetic_mi_eeg
from bci_framework.pipelines import PipelineRegistry
from bci_framework.agent import PipelineSelectionAgent
from bci_framework.logging import SnapshotLogger
from bci_framework.gui import build_leaderboard_table

## 1. Config and seed

In [ ]:
load_config(ROOT / "bci_framework" / "config.yaml")
config = get_config()
set_seed(config.get("experiment", {}).get("seed", 42))
print("Experiment ID:", get_experiment_id())

## 2. Load data (synthetic for demo)

In [ ]:
X, y = generate_synthetic_mi_eeg(n_trials=80, n_channels=22, n_samples=500, n_classes=4, fs=250.0, random_state=42)
print("Data shape:", X.shape, "Labels:", y.shape)
train_idx, test_idx = get_train_test_trials(len(X), train_ratio=0.8, random_state=42)
X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]
print("Train:", X_train.shape[0], "Test:", X_test.shape[0], "(trial-level, no leakage)")

## 3. Build pipelines and run calibration

In [ ]:
registry = PipelineRegistry(config)
pipelines = registry.build_pipelines(fs=250.0, n_classes=4)
pipelines = pipelines[:6]
agent = PipelineSelectionAgent(config)
metrics = agent.run_calibration(pipelines, X_train, y_train, n_classes=4)
kept = agent.prune(pipelines)
agent.select_top_n(kept)
best = agent.select_best()
print("Best pipeline:", best.name if best else None)

## 4. Leaderboard (sortable table)

In [ ]:
table = build_leaderboard_table(
    agent.get_metrics_dict(),
    top_n=10,
    sort_by="accuracy",
    selected_name=best.name if best else None,
)
import pandas as pd
df = pd.DataFrame(table)
display(df)

## 5. Snapshot saving

In [ ]:
results_dir = ROOT / "results" / get_experiment_id()
results_dir.mkdir(parents=True, exist_ok=True)
snapshot = SnapshotLogger(results_dir=results_dir)
for pipe in pipelines:
    if not getattr(pipe, "_fitted", False):
        continue
    snapshot.save_raw_eeg_plot(pipe.name, X_train[:1], fs=250.0)
    pred = pipe.predict(X_test)
    snapshot.save_confusion_matrix(pipe.name, y_test, pred)
    m = agent.get_metrics_dict().get(pipe.name, {})
    snapshot.save_json_log(pipe.name, m, selected=(best and pipe.name == best.name))
print("Snapshots saved to:", results_dir)

## 6. Optional: CSP explainability placeholder

In [ ]:
from bci_framework.gui import plot_csp_patterns
if best and hasattr(best.feature_extractor, "_filters") and best.feature_extractor._filters is not None:
    W = best.feature_extractor._filters
    fig = plot_csp_patterns(W, [f"Ch{i}" for i in range(W.shape[0])], save_path=results_dir / "csp_patterns.png")
    if fig:
        fig.show()